In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)
wiki.cache()

PythonRDD[2] at RDD at PythonRDD.scala:48

In [2]:
#prepare list of stop words
with open('/datasets/stop_words_en.txt', 'r') as stop_file:
    stop_words_str=stop_file.read()
stop_words_list = re.split('\n', stop_words_str, flags=re.UNICODE)

In [3]:
def filter_list(stop_list, listOfWords):
    filteredList = []
    for word in listOfWords:
        if is_word_in_stop_list(stop_list, word):
            continue
        else:
            filteredList.append(word.lower())
    return filteredList        

def is_word_in_stop_list(stop_list, word):
    if word.lower() in stop_list:
        return True
    else:
        return False
filtered = wiki.map(lambda x: filter_list(stop_words_list, x))    

In [6]:
def getBiGramsFromList(listOfWords):
    bigrams = []
    i = 0
    while i < len(listOfWords):
        if i == (len(listOfWords) - 1):
            return bigrams
        j = i + 1
        bigrams.append(listOfWords[i] + '_' + listOfWords[j])
        i += 1

counted_bigrams = filtered.flatMap(getBiGramsFromList).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

In [7]:
counted_bigrams.count()

4813006

In [8]:
counted_bigrams.cache()

PythonRDD[8] at RDD at PythonRDD.scala:48

In [10]:
numb_filtered = counted_bigrams.filter(lambda x: True if x[1] > 500 else False).cache()

In [11]:
numb_filtered.count()

39

In [12]:
numb_filtered.take(5)

[(u'york_city', 625),
 (u'notes_references', 638),
 (u'north_american', 510),
 (u'new_york', 4193),
 (u'roman_catholic', 630)]

In [13]:
def mapWordForCounting(listOfWords):
    result = []
    for word in listOfWords:
        result.append((word, 1))
    return result    

word_count = filtered.flatMap(mapWordForCounting).reduceByKey(lambda x, y: x+y).cache()

In [14]:
word_count.count()

426241

In [16]:
word_count.take(10)

[(u'biennials', 10),
 (u'underlyingly', 1),
 (u'ancyra', 43),
 (u'tripolitan', 2),
 (u'tilton', 4),
 (u'nordland', 1),
 (u'squealer', 8),
 (u'regularize', 2),
 (u'skylights.passive', 1),
 (u'thesis"(kleene', 1)]